<a href="https://colab.research.google.com/github/Nicolenki7/Online-Retail-Business-Analysis/blob/main/Online_Retail_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import sqlite3
from datetime import datetime
import numpy as np

# --- 0. Configuración y Carga Inicial ---
input_file_name = 'online_retail.csv'
table_name = 'RetailCleaned'
output_rfm_file = 'rfm_scores.csv'

try:
    # Cargar el dataset original
    df = pd.read_csv(input_file_name, encoding='ISO-8859-1')
    print("--- 1. Dataset original cargado exitosamente. ---")
    print(f"Filas iniciales: {len(df)}")
except FileNotFoundError:
    print(f"ERROR: Archivo '{input_file_name}' no encontrado. Por favor, súbelo a Colab.")
    exit()

# --- 2. Limpieza de Datos (Python/Pandas) ---

# 2.1. Crear la columna de Ingresos Totales (Sales_Total)
df['Sales_Total'] = df['Quantity'] * df['UnitPrice']

# 2.2. Filtrar transacciones no válidas/errores
df_cleaned = df[df['UnitPrice'] > 0]
df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]
df_cleaned.drop_duplicates(inplace=True)

# 2.3. Manejar Valores Nulos
df_cleaned.dropna(subset=['CustomerID'], inplace=True)

# 2.4. Conversión de Tipos y Preparación de Fechas
df_cleaned['InvoiceDate_dt'] = pd.to_datetime(df_cleaned['InvoiceDate'])
df_cleaned['CustomerID'] = df_cleaned['CustomerID'].astype(int).astype(str)
print(f"Filas después de Limpieza y Filtrado: {len(df_cleaned)}")

# --- 3. Cálculo de la Recencia (R) - ¡MIGRADO A PYTHON! ---

# 3.1. Definir la fecha de referencia
snapshot_date = df_cleaned['InvoiceDate_dt'].max() + pd.Timedelta(days=1)

# 3.2. Calcular la última fecha de compra para cada cliente
df_recency = df_cleaned.groupby('CustomerID')['InvoiceDate_dt'].max().reset_index()
df_recency.rename(columns={'InvoiceDate_dt': 'Last_Purchase_Date'}, inplace=True)

# 3.3. Calcular la Recencia en días
df_recency['Recency'] = (snapshot_date - df_recency['Last_Purchase_Date']).dt.days

# Limitar las columnas al resultado R (CustomerID, Recency)
df_recency = df_recency[['CustomerID', 'Recency']]

print(f"\n--- 3. Recencia (R) calculada en Python. ---")


# --- 4. Feature Engineering (SQL/SQLite) - Cálculo de F y M ---

# 4.1. Conexión y Carga de Datos a SQLite (solo para F y M)
conn = sqlite3.connect(':memory:')
# Usamos solo las columnas necesarias para F y M
df_cleaned[['CustomerID', 'InvoiceNo', 'Sales_Total']].to_sql(
    table_name, conn, if_exists='replace', index=False
)
print(f"\n--- 4. Datos cargados a la tabla SQLite '{table_name}'. ---")

# 4.2. Consulta SQL SIMPLE para Frecuencia y Valor Monetario
sql_query = f"""
SELECT
    CustomerID,
    -- FRECUENCIA (F): Número total de facturas únicas.
    COUNT(DISTINCT InvoiceNo) AS Frequency,

    -- VALOR MONETARIO (M): Suma total del gasto del cliente.
    ROUND(SUM(Sales_Total), 2) AS Monetary
FROM
    {table_name}
GROUP BY
    CustomerID
ORDER BY
    Monetary DESC;
"""

# Ejecutar la consulta y guardar los resultados
df_fm = pd.read_sql_query(sql_query, conn)
conn.close()
print("\n--- Frecuencia (F) y Monetario (M) calculados en SQL. ---")


# --- 5. Combinar RFM y Exportar ---

# 5.1. Combinar Recencia (R) de Python con Frecuencia y Monetario (F, M) de SQL
df_rfm = df_recency.merge(df_fm, on='CustomerID')

# 5.2. Exportación del Resultado RFM
df_rfm.to_csv(output_rfm_file, index=False)

print("\n=============================================")
print(f"✅ ¡FLUJO COMPLETO! El archivo '{output_rfm_file}' (RFM Scores) está listo.")
print("Primeras filas del resultado RFM:")
print(df_rfm.head())
print("=============================================")

--- 1. Dataset original cargado exitosamente. ---
Filas iniciales: 541909
Filas después de Limpieza y Filtrado: 392692

--- 3. Recencia (R) calculada en Python. ---

--- 4. Datos cargados a la tabla SQLite 'RetailCleaned'. ---

--- Frecuencia (F) y Monetario (M) calculados en SQL. ---

✅ ¡FLUJO COMPLETO! El archivo 'rfm_scores.csv' (RFM Scores) está listo.
Primeras filas del resultado RFM:
  CustomerID  Recency  Frequency  Monetary
0      12346      326          1  77183.60
1      12347        2          7   4310.00
2      12348       75          4   1797.24
3      12349       19          1   1757.55
4      12350      310          1    334.40


In [10]:
import pandas as pd
import numpy as np

# --- 1. Carga del Dataset RFM Scores ---
input_rfm_file = 'rfm_scores.csv'
try:
    df_rfm = pd.read_csv(input_rfm_file)
    print("--- 1. Dataset RFM Scores cargado exitosamente. ---")
except FileNotFoundError:
    print(f"ERROR: Archivo '{input_rfm_file}' no encontrado. Asegúrate de exportarlo correctamente en la celda anterior.")
    exit()

# 2. Análisis Descriptivo Inicial y Definición de Límites (Bins)
print("\n--- 2. Cuartiles de las Métricas RFM ---")
quantiles = df_rfm[['Recency', 'Frequency', 'Monetary']].quantile(q=[0.25, 0.5, 0.75]).T
print(quantiles)

# Definición de labels
r_labels = [4, 3, 2, 1]  # R: Recencia. Cuanto MÁS BAJA (más reciente), MEJOR (4).
f_m_labels = [1, 2, 3, 4] # F, M: Cuanto MÁS ALTO (mayor gasto/frecuencia), MEJOR (4).

# --- 3. Definición de Bins ROBUSTOS (Solución al ValueError) ---

# Crear límites de bins para R, F y M basados en los cuartiles + un límite extremo para evitar NaN
# Recencia Bins:
r_q = df_rfm['Recency'].quantile([0.25, 0.5, 0.75]).tolist()
r_bins = [-1, r_q[0], r_q[1], r_q[2], df_rfm['Recency'].max() + 1]

# Frequency Bins:
f_q = df_rfm['Frequency'].quantile([0.25, 0.5, 0.75]).tolist()
f_bins = [-1, f_q[0], f_q[1], f_q[2], df_rfm['Frequency'].max() + 1]

# Monetary Bins:
m_q = df_rfm['Monetary'].quantile([0.25, 0.5, 0.75]).tolist()
m_bins = [-1, m_q[0], m_q[1], m_q[2], df_rfm['Monetary'].max() + 1]

print("\n--- 3. Bins Robustos definidos. ---")

# --- 4. Asignación de Puntuación (Scoring) ---

# Usar pd.cut con los límites explícitos (derecha=False para incluir el límite inferior)
df_rfm['R_Score'] = pd.cut(df_rfm['Recency'], bins=r_bins, labels=r_labels, include_lowest=True, right=False).astype(float)
df_rfm['F_Score'] = pd.cut(df_rfm['Frequency'], bins=f_bins, labels=f_m_labels, include_lowest=True, right=False).astype(float)
df_rfm['M_Score'] = pd.cut(df_rfm['Monetary'], bins=m_bins, labels=f_m_labels, include_lowest=True, right=False).astype(float)

# 4.1. Gestión de NaN y Conversión a Entero (¡CRÍTICO!)
# Reemplazar cualquier posible NaN con 1.0 (puntuación mínima) antes de la conversión a entero.
df_rfm['R_Score'].fillna(1.0, inplace=True)
df_rfm['F_Score'].fillna(1.0, inplace=True)
df_rfm['M_Score'].fillna(1.0, inplace=True)

# Convertir scores a enteros
df_rfm['R_Score'] = df_rfm['R_Score'].astype(int)
df_rfm['F_Score'] = df_rfm['F_Score'].astype(int)
df_rfm['M_Score'] = df_rfm['M_Score'].astype(int)

# Crear la Puntuación RFM Combinada (ej. 434)
df_rfm['RFM_Score'] = df_rfm['R_Score'].astype(str) + df_rfm['F_Score'].astype(str) + df_rfm['M_Score'].astype(str)
print("\n--- 4. Puntuaciones RFM Creadas y Estabilizadas. ---")
print(df_rfm[['CustomerID', 'Recency', 'R_Score', 'F_Score', 'M_Score', 'RFM_Score']].head())


# --- 5. Segmentación de Negocio (Categorización) ---

# Definición de los Segmentos de Negocio
segment_rules = {
    'Campeones': r'[4-5][4-5][4-5]',
    'Clientes Leales': r'[3-5][3-5][2-5]',
    'Potencial Leal': r'[3-4][1-3][1-3]',
    'Nuevos Clientes': r'[4-5][1-2][1-2]',
    'Clientes en Riesgo': r'[1-2][3-5][3-5]',
    'No se pueden perder': r'[1-2][4-5][4-5]',
    'Durmiendo': r'[1-2][1-2][1-2]',
}

df_rfm['Segment'] = 'Otros'

# Aplicar las reglas de segmentos
for segment, pattern in segment_rules.items():
    df_rfm.loc[df_rfm['RFM_Score'].str.match(pattern), 'Segment'] = segment

# --- 6. Exportación del Resultado Final (Ready for BI Tool) ---
output_final_file = 'rfm_segmentation_final.csv'
df_rfm.to_csv(output_final_file, index=False)

print("\n--- 5. Segmentación Finalizada ---")
print(df_rfm['Segment'].value_counts())
print(f"\n✅ ¡Fase 5 Completa! El archivo '{output_final_file}' está listo para Power BI/Looker Studio.")

--- 1. Dataset RFM Scores cargado exitosamente. ---

--- 2. Cuartiles de las Métricas RFM ---
               0.25    0.50       0.75
Recency     18.0000   51.00   142.0000
Frequency    1.0000    2.00     5.0000
Monetary   306.4825  668.57  1660.5975

--- 3. Bins Robustos definidos. ---

--- 4. Puntuaciones RFM Creadas y Estabilizadas. ---
   CustomerID  Recency  R_Score  F_Score  M_Score RFM_Score
0       12346      326        1        2        4       124
1       12347        2        4        4        4       444
2       12348       75        2        3        4       234
3       12349       19        3        2        4       324
4       12350      310        1        2        2       122

--- 5. Segmentación Finalizada ---
Segment
Clientes Leales        1050
Durmiendo              1015
Potencial Leal         1013
Otros                   566
Clientes en Riesgo      489
No se pueden perder     117
Nuevos Clientes          88
Name: count, dtype: int64

✅ ¡Fase 5 Completa! El archivo '

/tmp/ipython-input-4194371863.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_rfm['R_Score'].fillna(1.0, inplace=True)
/tmp/ipython-input-4194371863.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using